In [1]:
import sagemaker
from sagemaker import get_execution_role

role = get_execution_role()
print(role)
sess = sagemaker.Session()

arn:aws:iam::117588387775:role/service-role/AmazonSageMaker-ExecutionRole-20190513T204887


In [2]:
from sagemaker.amazon.amazon_estimator import get_image_uri

training_image = get_image_uri(sess.boto_region_name, 'object-detection', repo_version='latest')
print (training_image)

825641698319.dkr.ecr.us-east-2.amazonaws.com/object-detection:latest


In [3]:
bucket = 'sagemaker-turi-create' # custom bucket name.
train_channel = 'train'
validation_channel = 'validation'
s3_output_location = 's3://{}/output'.format(bucket)

In [4]:
od_model = sagemaker.estimator.Estimator(training_image,
                                         role, 
                                         train_instance_count=1, 
                                         train_instance_type='ml.p3.2xlarge',
                                         train_volume_size = 50,
                                         train_max_run = 360000,
                                         input_mode= 'File',
                                         output_path=s3_output_location,
                                         sagemaker_session=sess)

In [5]:
def set_hyperparameters(num_epochs, lr_steps):
    num_classes = 16#classes_df.shape[0]
    num_training_samples = 66#train_df.shape[0]
    print('num classes: {}, num training images: {}'.format(num_classes, num_training_samples))

    od_model.set_hyperparameters(base_network='resnet-50',
                                 use_pretrained_model=1,
                                 num_classes=num_classes,
                                 mini_batch_size=16,
                                 epochs=num_epochs,               
                                 learning_rate=0.001, 
                                 lr_scheduler_step=lr_steps,      
                                 lr_scheduler_factor=0.1,
                                 optimizer='sgd',
                                 momentum=0.9,
                                 weight_decay=0.0005,
                                 overlap_threshold=0.5,
                                 nms_threshold=0.45,
                                 image_shape=512,
                                 label_width=350,
                                 num_training_samples=num_training_samples)

In [6]:
set_hyperparameters(100, '33,67')

num classes: 16, num training images: 66


In [7]:
s3_train_data = 's3://{}/{}'.format(bucket, train_channel)
s3_validation_data = 's3://{}/{}'.format(bucket, validation_channel)

train_data = sagemaker.session.s3_input(s3_train_data, distribution='FullyReplicated', 
                        content_type='application/x-recordio', s3_data_type='S3Prefix')
validation_data = sagemaker.session.s3_input(s3_validation_data, distribution='FullyReplicated', 
                             content_type='application/x-recordio', s3_data_type='S3Prefix')
data_channels = {'train': train_data, 'validation': validation_data}

In [ ]:
%%time
od_model.fit(inputs=data_channels, logs=True)

2019-07-18 04:22:47 Starting - Starting the training job...
2019-07-18 04:22:49 Starting - Launching requested ML instances......
2019-07-18 04:23:52 Starting - Preparing the instances for training...
2019-07-18 04:24:44 Downloading - Downloading input data...
2019-07-18 04:25:05 Training - Downloading the training image.........
2019-07-18 04:26:42 Training - Training image download completed. Training in progress.
Docker entrypoint called with argument(s): train
[07/18/2019 04:26:44 INFO 140536547198784] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/default-input.json: {u'label_width': u'350', u'early_stopping_min_epochs': u'10', u'epochs': u'30', u'overlap_threshold': u'0.5', u'lr_scheduler_factor': u'0.1', u'_num_kv_servers': u'auto', u'weight_decay': u'0.0005', u'mini_batch_size': u'32', u'use_pretrained_model': u'0', u'freeze_layer_pattern': u'', u'lr_scheduler_step': u'', u'early_stopping': u'False', u'early_stopping_patience': u'5', u'mome